# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/bidding-3233000
INFO:tensorflow:Restoring parameters from ..\Models/binfo-808000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, True

# you sit West and hold:
hand = 'AKQJ95.J54..K953'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "2D", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': 'PASS', 'insta_score': 0.9765}]

In [4]:
auction = ["PAD_START", "2D", "PASS", "PASS", "X", "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': '2H', 'insta_score': 0.007, 'expected_score': 45, 'adjustment': 0},
 {'call': 'PASS',
  'insta_score': 0.9783,
  'expected_score': -41,
  'adjustment': 49}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['8xxx.KQxx.A.AQxx Tx.A9x.KQxxxx.Tx x.T8x.JT98xx.J8x AKQJ9x.Jxx..K9xx 0.80416 2D-P-P-X-P-P-P (7.8)  2D-P-P-X-P-P-2H-P-P-P (8.1) ',
 'Txx.A8xx.AJ.AQ8x x.KQT.K98xxx.Txx 8xx.9xx.QTxxx.Jx AKQJ9x.Jxx..K9xx 0.75914 2D-P-P-X-P-P-P (7.6)  2D-P-P-X-P-P-2H-P-P-P (8.9) ',
 'xxx.AKQx.Jx.Axxx x.9xx.AK9xxx.QTx T8x.T8x.QT8xx.J8 AKQJ9x.Jxx..K9xx 0.74239 2D-P-P-X-P-P-P (7.7)  2D-P-P-X-P-P-2H-P-P-P (8.0) ',
 'xxxx.KQxx.KT.AQx 8.Txx.AQJ9xx.xxx Tx.A98.8xxxx.JT8 AKQJ9x.Jxx..K9xx 0.67475 2D-P-P-X-P-P-P (7.7)  2D-P-P-X-P-P-2H-P-P-P (7.5) ',
 'Txx.AK9x.AQx.AQx xx.Qxx.KJ9xxx.8x 8x.T8x.T8xx.JTxx AKQJ9x.Jxx..K9xx 0.62271 2D-P-P-X-P-P-P (6.5)  2D-P-P-X-P-P-2H-X-P-3C-P-P-P (6.7) ',
 'T8x.AK9x.Qx.AJxx x.8xx.AJT8xx.QT8 xxx.QTx.K9xxx.xx AKQJ9x.Jxx..K9xx 0.59671 2D-P-P-X-P-P-P (7.3)  2D-P-P-X-P-P-2H-P-P-P (7.6) ',
 'x.AKQ98.Axx.AJ8x xxx.xx.KQT9xx.Qx T8x.Txx.J8xx.Txx AKQJ9x.Jxx..K9xx 0.57238 2D-P-P-X-P-P-P (7.6)  2D-P-P-X-P-P-2H-P-P-P (7.5) ',
 'Txx.AK9x.Kx.QT8x 8x.Txx.QJ98xx.AJ xx.Q8x.ATxxx.xxx AKQJ9x.Jxx..K9xx 0.5147

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

AKQJ9x.Jxx..K9xx Txx.K8x.J8xxx.xx 8.T9xx.KQ9x.AQTx xxx.AQx.ATxx.J8x
AKQJ9x.Jxx..K9xx Txx.A9x.AJTxx.8x x.Q8xx.KQ8x.QJxx 8xx.KTx.9xxx.ATx
AKQJ9x.Jxx..K9xx xx.AKQ9.9xx.QT8x x.T8xx.AKJx.Axxx T8xx.xx.QT8xxx.J
AKQJ9x.Jxx..K9xx Txx.QT9x.Axxx.Jx .K8xx.Jxx.AQT8xx 8xxx.Ax.KQT98x.x
AKQJ9x.Jxx..K9xx 8xx.8x.QJxxx.AJ8 x.QTxx.AK8xx.Qxx Txx.AK9x.T9x.Txx
AKQJ9x.Jxx..K9xx 8xx.QT9.QT8xx.Jx x.Kxxx.A9xx.AQxx Txx.A8x.KJxx.T8x
AKQJ9x.Jxx..K9xx xxx.8xx.A8xx.AQx x.AQTx.KQxx.JTxx T8x.K9x.JT9xx.8x
AKQJ9x.Jxx..K9xx Txx.AQ8x.QJx.A8x x.KTxx.A8xxx.QTx 8xx.9x.KT9xx.Jxx
AKQJ9x.Jxx..K9xx 8x.Q8x.KJTxx.Txx x.AKxx.9xxx.QJxx Txxx.T9x.AQ8x.A8
AKQJ9x.Jxx..K9xx xxxx.K9.AQTxxx.x T.A8xx.K9xx.QT8x 8x.QTxx.J8x.AJxx
AKQJ9x.Jxx..K9xx 8xxx.9xx.AK9x.8x x.KQTx.QJxx.AQxx Tx.A8x.T8xxx.JTx
AKQJ9x.Jxx..K9xx 8xxx.KT8.QJ9x.Tx x.AQxx.Axxx.J8xx Tx.9xx.KT8xx.AQx
AKQJ9x.Jxx..K9xx Txxx.9x.AKJ9.JTx x.AQTx.Q8x.AQxxx 8x.K8xx.Txxxxx.8
AKQJ9x.Jxx..K9xx T8x.K8x.K8x.Q8xx x.9xxx.AQxxx.ATx xxx.AQT.JT9xx.Jx
AKQJ9x.Jxx..K9xx xx.K9x.K9xxx.JTx T.QT8x.AT8xx.A